In [1]:
%load_ext autoreload
%autoreload 2



In [2]:
import torch
import pyro
from pyro.optim import Adam, ClippedAdam
import congas as cg
from congas.models import LatentCategorical
from pyro.infer import TraceMeanField_ELBO,TraceEnum_ELBO, TraceGraph_ELBO, Trace_ELBO
torch.set_default_tensor_type('torch.cuda.FloatTensor')

/home/salvatore.milite/miniconda3/envs/scdeepaa/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pickle

data_file = open("data.pkl",'rb')

data = pickle.loads(data_file.read())
data_file.close()

param_file =  open("parameters.pkl",'rb')

param = pickle.loads(param_file.read())
param_file.close()


interface = cg.Interface(CUDA = True)

In [4]:
param

{'probs': tensor([0.2000, 0.6000, 0.0500, 0.0250, 0.0250], device='cpu'),
 'init_probs': 0.6,
 'purity': 0.48,
 'theta_shape_atac': tensor([12.4963, 16.6187,  4.3866,  9.7522], device='cpu'),
 'theta_rate_atac': tensor([0.0135, 0.0067, 0.0125, 0.0125], device='cpu'),
 'nb_size_init_atac': tensor([150., 150., 150., 150.], device='cpu'),
 'likelihood_atac': 'NB',
 'a': 0.1,
 'b': 100.0,
 'lambda': 0.0,
 'hidden_dim': 5,
 'binom_prior_limits': [40.0, 1000.0],
 'equal_sizes_sd': True,
 'Temperature': 20.0,
 'K': [1, 2, 3],
 'latent_type': 'G'}

In [5]:
sim_data = cg.simulate_data_congas()
sim_data

{'atac': {'exp': array([[ 235., 1048.,  268., ...,  437.,  650.,  872.],
         [ 232.,  481.,  187., ...,  231.,  164.,  193.],
         [ 682.,  181.,  677., ...,  156.,  107.,  550.],
         ...,
         [  57.,  112.,  130., ...,  101.,  386.,   55.],
         [ 223.,  147.,  154., ...,  274.,  213.,  455.],
         [ 389.,  107.,  201., ...,  139.,  212.,  225.]]),
  'MAF': array([[0.25757086, 0.43895297, 0.39015632, ..., 0.50143799, 0.40552971,
          0.45961821],
         [0.38259681, 0.48456466, 0.37160218, ..., 0.22721365, 0.35204687,
          0.46875249],
         [0.51091586, 0.37702521, 0.45929178, ..., 0.41399496, 0.42624313,
          0.28206325],
         ...,
         [0.32702018, 0.34648236, 0.31799088, ..., 0.25316452, 0.33604643,
          0.33607663],
         [0.41066895, 0.35950444, 0.29118441, ..., 0.28865367, 0.30410748,
          0.36543362],
         [0.2412723 , 0.33319388, 0.3266506 , ..., 0.44785145, 0.22827509,
          0.31391053]]),
  'labels'

In [6]:
data = {
    "data_atac" : torch.tensor(sim_data["atac"]["exp"]).T.float(),
    "data_rna" : torch.tensor(sim_data["rna"]["exp"]).T.float(),
    "norm_factor_atac" : torch.tensor(sim_data["atac"]["norm_factors"]).float(),
    "norm_factor_rna" : torch.tensor(sim_data["rna"]["norm_factors"]).float(),
    "pld" : torch.tensor(sim_data['CNA_bulk']['total'].values).float(), 
}

In [7]:
param["binom_prior_limits"] = torch.tensor([10,1000])
param["theta_shape_atac"] = torch.tensor(sim_data["atac"]["theta_shape_atac"]).float()
param["theta_rate_atac"] = torch.tensor(sim_data["atac"]["theta_rate_atac"]).float()
param["theta_shape_rna"] = torch.tensor(sim_data["rna"]["theta_shape_rna"]).float()
param["theta_rate_rna"] = torch.tensor(sim_data["rna"]["theta_rate_rna"]).float()
param['nb_size_init_atac'] = torch.tensor([150., 150., 150., 150.])
param["K"] = 1
param['probs'] = torch.tensor([0.2000, 0.6000, 0.0500, 0.0250, 0.0250])
param["likelihood_atac"] = "NB"
param["likelihood_rna"] = "NB"
param["temperature"] = 10
param["lambda"] = 1
param["purity"] = None
param["nb_size_init_atac"] = torch.ones(10) * 150
param["nb_size_init_rna"] = torch.ones(10) * 150
param["CUDA"] = True


In [8]:
data

{'data_atac': tensor([[ 235.,  232.,  682.,  ...,   57.,  223.,  389.],
         [1048.,  481.,  181.,  ...,  112.,  147.,  107.],
         [ 268.,  187.,  677.,  ...,  130.,  154.,  201.],
         ...,
         [ 437.,  231.,  156.,  ...,  101.,  274.,  139.],
         [ 650.,  164.,  107.,  ...,  386.,  213.,  212.],
         [ 872.,  193.,  550.,  ...,   55.,  455.,  225.]]),
 'data_rna': tensor([[ 731.,  258.,  387.,  ...,  142.,  162.,   96.],
         [ 551.,  362.,  663.,  ...,  230.,  134.,  454.],
         [ 314.,   57.,  879.,  ...,  182.,   27.,   46.],
         ...,
         [ 280.,   68., 2098.,  ...,   64.,  210.,  106.],
         [ 164., 1871., 1277.,  ...,  131.,  240.,  165.],
         [1319., 1518., 1020.,  ...,   46.,   73.,  201.]]),
 'norm_factor_atac': tensor([ 2.5419, 10.0350,  3.0248,  ...,  2.7805,  2.7843,  2.9520]),
 'norm_factor_rna': tensor([5.2224, 2.9358, 1.7162,  ..., 1.9030, 4.8586, 5.8741]),
 'pld': tensor([4., 3., 2., 4., 2., 2., 3., 4., 3., 3.])}

In [9]:
interface.set_model(LatentCategorical)
interface.set_optimizer(ClippedAdam)
interface.set_loss(Trace_ELBO)
interface.initialize_model(data)
interface.set_model_params(param)


In [10]:
ll = interface.run(steps = 200, param_optimizer = {"lr":0.1})



ELBO: 3.419012395  : 100%|██████████| 200/200 [00:02<00:00, 82.74it/s]


Done!


In [12]:

import numpy as np


lr = interface.learned_parameters()
ICs = interface.calculate_ICs()


Computing assignment probabilities
Computing information criteria.
tensor(-205154.0625, grad_fn=<SumBackward0>)
tensor(-1101797.6250, grad_fn=<SumBackward0>)


In [ ]:

data["data_atac"][0,:].mean()

In [ ]:
from congas import log_sum_exp
lk_atac = lk_atac + torch.log(lr_t["mixture_weights_atac"]).reshape([2,1,1])

In [ ]:
torch.exp(lk_atac - log_sum_exp(lk_atac))

In [ ]:
np.sum(lr["assignment_atac"] == 0)

In [ ]:
data